In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.mft import Mft
from checklist.inv_dir import Inv, Dir
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.perturb import Perturb


In [2]:
import sys
sys.path.append('/home/marcotcr/work/ml-tests/')
from mltests import bert_squad_model
from checklist.pred_wrapper import PredictorWrapper
model = bert_squad_model.BertSquad()
invert = lambda a: model.predict_pairs([(x[1], x[0]) for x in a])
new_pp = PredictorWrapper.wrap_predict(invert)

In [3]:
model.predict_pairs([('Who is smarter?', 'John is smart')])

['John']

In [4]:
editor = checklist.editor.Editor()
editor.tg

In [293]:
nlp = spacy.load('en_core_web_sm')

In [5]:
def format_squad_with_context(x, pred, conf, label=None, *args, **kwargs):
    c, q = x
    ret = 'C: %s\nQ: %s\n' % (c, q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret

In [6]:
def format_squad(x, pred, conf, label=None, *args, **kwargs):
    c, q = x
    ret = 'Q: %s\n' % (q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret

In [268]:
import json
def load_squad(fold='validation'):
    answers = []
    data = []
    ids = []
    files = {
        'validation': '/home/marcotcr/datasets/squad/dev-v1.1.json',
        'train': '/home/marcotcr//datasets/squad/train-v1.1.json',
        }
    f = json.load(open(files[fold]))
    for t in f['data']:
        for p in t['paragraphs']:
            context = p['context']
            for qa in p['qas']:
                data.append({'passage': context, 'question': qa['question'], 'id': qa['id']})
                answers.append(set([(x['text'], x['answer_start']) for x in qa['answers']]))
    return data, answers


In [270]:
import pickle
data, answers =  load_squad()
spacy_map =  pickle.load(open('/home/marcotcr/tmp/processed_squad.pkl', 'rb'))
pairs = [(x['passage'], x['question']) for x in data]
processed_pairs = [(spacy_map[x[0]], spacy_map[x[1]]) for x in pairs]

## Vocabulary

In [7]:
print(', '.join(editor.suggest('{first_name} is {bert} than {first_name2}.')[:60]))

better, older, worse, smarter, taller, younger, different, more, stronger, bigger, shorter, less, tougher, cooler, faster, nicer, other, darker, hotter, smaller, happier, larger, wiser, richer, weaker, greater, higher, longer, harder, safer, closer, heavier, slower, easier, colder, thinner, quicker, thicker, deeper, quieter, lower, healthier, stranger, louder, cleaner, brighter, lighter, cheaper, newer, simpler, warmer, wealthier, poorer, sharper, wider, smoother, clearer, softer, superior, stricter


In [8]:
adj = ['old', 'smart', 'tall', 'young', 'strong', 'short', 'tough', 'cool', 'fast', 'nice', 'small', 'dark', 'wise', 'rich', 'great', 'weak', 'high', 'slow', 'strange', 'clean']
adj = [(x.rstrip('e'), x) for x in adj]


In [9]:
data, labels = map(list, zip(*(editor.template(
    ((
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is less {adj[1]}?'
    ),
    '{first_name}',
    ),
    adj=adj,
    remove_duplicates=True,
    nsamples=200,
    ))))
test = Mft(data, labels=labels)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)

Predicting 198 examples



Test cases:      198
Fails (rate):    158 (79.8%)

Example fails:
C: Jessica is cleaner than Joshua.
Q: Who is less clean?
A: Jessica
P: Joshua

----
C: Olivia is cooler than Heather.
Q: Who is less cool?
A: Olivia
P: Heather

----
C: Justin is nicer than William.
Q: Who is less nice?
A: Justin
P: William

----


In [10]:
comp_pairs = [('better', 'worse'), ('older', 'younger'), ('smarter', 'dumber'), ('taller', 'shorter'), ('bigger', 'smaller'), ('stronger', 'weaker'), ('faster', 'slower'), ('darker', 'lighter'), ('richer', 'poorer'), ('happier', 'sadder'), ('louder', 'quieter'), ('warmer', 'colder')]
comp_pairs = list(set(comp_pairs + [(x[1], x[0]) for x in comp_pairs]))

In [11]:
data, labels = map(list, zip(*(editor.template(
    ((
    '{first_name} is {comp[0]} than {first_name1}.',
    'Who is {comp[1]}?'
    ),
    '{first_name}',
    ),
    comp=comp_pairs,
    remove_duplicates=True,
    nsamples=200,
    ))))
test = Mft(data, labels=labels)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)

Predicting 199 examples



Test cases:      199
Fails (rate):    120 (60.3%)

Example fails:
C: Emma is slower than Jennifer.
Q: Who is faster?
A: Emma
P: Jennifer

----
C: William is smaller than Kyle.
Q: Who is bigger?
A: William
P: Kyle

----
C: Christian is stronger than Brittany.
Q: Who is weaker?
A: Christian
P: Brittany

----


In [12]:
state = editor.suggest('John is very {bert} about the project.')[:20]
print(', '.join(editor.suggest('John is {bert} {state} about the project.', state=state)[:30]))
very = ['very', 'extremely', 'really', 'quite', 'incredibly', 'particularly', 'highly', 'super']
somewhat = ['a little', 'somewhat', 'slightly', 'mildly']

very, extremely, quite, also, not, pretty, really, still, more, incredibly, particularly, clearly, so, now, highly, already, rather, fairly, most, certainly, being, definitely, super, increasingly, especially, generally, actually, reportedly, apparently, understandably


In [13]:
def to_datalabels(templates, nsamples=None):
    ret = []
    ret_labels = []
    for x in templates:
        cs = x['contexts']
        qas = x['qas']
        d = list(itertools.product(cs, qas))
        ret.append([(x[0], x[1][0]) for x in d])
        ret_labels.append([x[1][1] for x in d])
    return ret, ret_labels

In [14]:
data, labels = to_datalabels(editor.template(
    {
        'contexts': [
            '{first_name} is {very} {s} about the project. {first_name1} is {s} about the project.',
            '{first_name1} is {s} about the project. {first_name} is {very} {s} about the project.',
            '{first_name} is {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {s} about the project.',
            '{first_name} is {very} {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {very} {s} about the project.',
        ],
        'qas': [
            (
                'Who is most {s} about the project?',
                '{first_name}'
            ), 
            (
                'Who is least {s} about the project?',
                '{first_name1}'
            ), 
            
        ]
        
    },
    s = state,
    very=very,
    somewhat=somewhat,
    remove_duplicates=True,
    nsamples=200,
    ))
test = Mft(data, labels=labels)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)


Predicting 2376 examples



Test cases:      198
Fails (rate):    180 (90.9%)

Example fails:
C: Jonathan is serious about the project. Charles is very serious about the project.
Q: Who is least serious about the project?
A: Jonathan
P: Charles


----
C: Samantha is confident about the project. Kyle is very confident about the project.
Q: Who is least confident about the project?
A: Samantha
P: Kyle

C: Samantha is somewhat confident about the project. Kyle is very confident about the project.
Q: Who is least confident about the project?
A: Samantha
P: Kyle


----
C: Brandon is pessimistic about the project. Zachary is particularly pessimistic about the project.
Q: Who is least pessimistic about the project?
A: Brandon
P: Zachary

C: Zachary is pessimistic about the project. Brandon is somewhat pessimistic about the project.
Q: Who is least pessimistic about the project?
A: Brandon
P: Zachary

C: Brandon is somewhat pessimistic about the project. Zachary is pessimistic about the project.
Q: Who is most pessimist

## Taxonomy

### Size, chape, color, age, material

In [15]:
import munch
order = ['size', 'shape', 'age', 'color']
props = []
properties = {
    'color' : ['red', 'blue','yellow', 'green', 'pink', 'white', 'black', 'orange', 'grey', 'purple', 'brown'],
    'size' : ['big', 'small', 'tiny', 'enormous'],
    'age' : ['old', 'new'],
    'shape' : ['round', 'oval', 'square', 'triangular'],
    'material' : ['iron', 'wooden', 'ceramic', 'glass', 'stone']
}
for i in range(len(order)):
    for j in range(i + 1, len(order)):
        p1, p2 = order[i], order[j]
        for v1, v2 in itertools.product(properties[p1], properties[p2]):
            props.append(munch.Munch({
                'p1': p1,
                'p2': p2,
                'v1': v1,
                'v2': v2,
            }))


In [16]:
print(', '.join(editor.suggest('There is {a:p.v1} {p.v2} {bert} in the room.', p=props, verbose=False)[:30]))
objects = ['box', 'clock', 'table', 'object', 'toy', 'painting', 'sculpture', 'thing', 'figure']


light, wall, table, object, couch, door, sofa, chair, mirror, box, window, clock, bar, elephant, lamp, carpet, curtain, room, bed, ball, spot, building, hole, stone, screen, candle, tree, cat, square, painting


In [17]:
data, labels = to_datalabels(editor.template(
    {
        'contexts': [
            'There is {a:p.v1} {p.v2} {obj} in the room.',
            'There is {a:obj} in the room. The {obj} is {p.v1} and {p.v2}.',
        ],
        'qas': [
            (
                'What {p.p1} is the {obj}?',
                '{p.v1}'
            ), 
            (
                'What {p.p2} is the {obj}?',
                '{p.v2}'
            ), 
            
        ]
        
    },
    obj=objects,
    p=props,
    remove_duplicates=True,
    nsamples=200,
    ))
test = Mft(data, labels=labels)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)

Predicting 800 examples



Test cases:      200
Fails (rate):    166 (83.0%)

Example fails:
C: There is a thing in the room. The thing is enormous and oval.
Q: What size is the thing?
A: enormous
P: enormous and oval


----
C: There is a round purple clock in the room.
Q: What shape is the clock?
A: round
P: round purple

C: There is a clock in the room. The clock is round and purple.
Q: What shape is the clock?
A: round
P: round and purple


----
C: There is a tiny black object in the room.
Q: What size is the object?
A: tiny
P: tiny black


----


### Professions vs nationalities

In [20]:
professions = editor.suggest('{first_name} works as {a:bert}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:bert}.')[:30]
professions = list(set(professions))
if 'translator' in professions:
    professions.remove('translator')

In [21]:
def clean(string):
    return string.lstrip('[a,the,an,in,at] ').rstrip('.')

In [22]:
def expect_squad(x, pred, conf, label=None, meta=None):
    return clean(pred) == clean(label)
expect_squad = Expect.single(expect_squad)

In [23]:
data, labels = to_datalabels(editor.template(
    {
        'contexts': [
            '{first_name} is {a:nat} {prof}.',
            '{first_name} is {a:prof}. {first_name} is {nat}.',
            '{first_name} is {nat}. {first_name} is {a:prof}.',
            '{first_name} is {nat} and {a:prof}.',
            '{first_name} is {a:prof} and {nat}.',
        ],
        'qas': [
            (
                'What is {first_name}\'s job?',
                '{prof}'
            ), 
            (
                'What is {first_name}\'s nationality?',
                '{nat}'
            ), 
            
        ]
        
    },
    nat = editor.lexicons['nationality'][:10],
    prof=professions,
    remove_duplicates=True,
    nsamples=100,
    ))
test = Mft(data, expect=expect_squad, labels=labels)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)

Predicting 1000 examples



Test cases:      100
Fails (rate):    51 (51.0%)

Example fails:
C: Danielle is a Russian author.
Q: What is Danielle's job?
A: author
P: Russian author


----
C: Emily is a Brazilian consultant.
Q: What is Emily's job?
A: consultant
P: Brazilian consultant


----
C: Elizabeth is a Russian salesman.
Q: What is Elizabeth's job?
A: salesman
P: Russian salesman


----


### Animal vs vehicle

In [24]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
data, labels = to_datalabels(editor.template(
    {
        'contexts': [
            '{first_name} has {a:animal} and {a:vehicle}.',
            '{first_name} has {a:vehicle} and {a:animal}.',
        ],
        'qas': [
            (
                'What animal does {first_name} have?',
                '{animal}'
            ), 
            (
                'What vehicle does {first_name} have?',
                '{vehicle}'
            ), 
            
        ]
        
    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=100,
    ))
test = Mft(data, expect=expect_squad, labels=labels)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)


Predicting 400 examples



Test cases:      100
Fails (rate):    21 (21.0%)

Example fails:
C: Michael has a bull and a train.
Q: What vehicle does Michael have?
A: train
P: a bull and a train

C: Michael has a train and a bull.
Q: What vehicle does Michael have?
A: train
P: a train and a bull


----
C: Michael has a bull and a tractor.
Q: What animal does Michael have?
A: bull
P: a bull and a tractor

C: Michael has a bull and a tractor.
Q: What vehicle does Michael have?
A: tractor
P: a bull and a tractor

C: Michael has a tractor and a bull.
Q: What animal does Michael have?
A: bull
P: a tractor and a bull


----
C: Alexander has a cow and a tractor.
Q: What vehicle does Alexander have?
A: tractor
P: a cow and a tractor

C: Alexander has a tractor and a cow.
Q: What animal does Alexander have?
A: cow
P: a tractor and a cow


----


## Robustness

typos

In [291]:
def question_typo(x):
    return (x[0], Perturb.add_typos(x[1]))
data = Perturb.perturb(pairs, question_typo, nsamples=300)
test = Inv(data)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad)

Predicting 600 examples



Test cases:      300
Fails (rate):    46 (15.3%)

Example fails:
C: Tesla gained experience in telephony and electrical engineering before emigrating to the United States in 1884 to work for Thomas Edison in New York City. He soon struck out on his own with financial backers, setting up laboratories and companies to develop a range of electrical devices. His patented AC induction motor and transformer were licensed by George Westinghouse, who also hired Tesla for a short time as a consultant. His work in the formative years of electric power development was involved in a corporate alternating current/direct current "War of Currents" as well as various patent battles.
Q: Where did Tesla work with Edison?
P: New York City

C: Tesla gained experience in telephony and electrical engineering before emigrating to the United States in 1884 to work for Thomas Edison in New York City. He soon struck out on his own with financial backers, setting up laboratories and companies to develop a range

Contractions

In [298]:
def contractions(x):
    conts = Perturb.contractions(x[1])
    return [(x[0], a) for a in conts]
data = Perturb.perturb(pairs, contractions, nsamples=300)
test = Inv(data)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad)

Test cases:      300
Fails (rate):    14 (4.7%)

Example fails:
Q: If someone is being taught at their place of residence, what is it called?
P: Informal learning

Q: If someone is being taught at their place of residence, what's it called?
P: home schooling


----
Q: How did Celeron handle meeting with Old Briton?
P: threatened "Old Briton" with severe consequences

Q: How'd Celeron handle meeting with Old Briton?
P: Céloron threatened "Old Briton" with severe consequences


----
Q: Where did the discharge from glaciers go in Europe in the last Ice Age?
P: the Rhine

Q: Where'd the discharge from glaciers go in Europe in the last Ice Age?
P: the Rhine and its downstream extension


----


Add random sentence

In [301]:
random_sentences = set()
for x, _ in processed_pairs:
    for y in x.sents:
        random_sentences.add(y.text)
random_sentences = list(random_sentences)

In [303]:
# len(random_sentences)

In [307]:
meta[0]

[{},
 'add to end: Later in the summer, Kenyatta visited China at the invitation of President Xi Jinping after a stop in Russia and not having visited the United States as president. ',
 'add to beg: Later in the summer, Kenyatta visited China at the invitation of President Xi Jinping after a stop in Russia and not having visited the United States as president. ']

In [309]:
def add_random_sentence(x):
    random_s = np.random.choice(random_sentences)
    while random_s in x[0]:
        random_s = np.random.choice(random_sentences)
    random_s = random_s.strip('.') + '. '
    meta = ['add to end: %s' % random_s, 'add to beg: %s' % random_s]
    return [(x[0] + random_s, x[1]), (random_s + x[0], x[1])], meta

def format_add(x, pred, conf, label=None, meta=None):
    ret = format_squad(x, pred, conf, label, meta)
    if meta:
        ret += 'Perturb: %s\n' % meta
    return ret

# data, meta = Perturb.perturb(pairs, add_random_sentence, nsamples=300, returns_meta=True)
# test = Inv(data, meta=meta)
# test.run(new_pp)
test.summary(n=3, format_example_fn=format_add)
    

Test cases:      300
Fails (rate):    18 (6.0%)

Example fails:
Q: What do most online pharmacies do?
P: sell prescription drugs

Q: What do most online pharmacies do?
P: sell prescription drugs and require a valid prescription
Perturb: add to beg: In 2005 it moved to a new facility on The Watermark business park next to the MetroCentre in Gateshead. 


----
Q: Concrete bounding of computation time frequently produces complexity classes contingent upon what?
P: the chosen machine model

Q: Concrete bounding of computation time frequently produces complexity classes contingent upon what?
P: machine model
Perturb: add to end: Many important complexity classes can be defined by bounding the time or space used by the algorithm. 


----
Q: What is formed when a phagosome fuses with a lysosome?
P: a phagolysosome

Q: What is formed when a phagosome fuses with a lysosome?
P: phagolysosome
Perturb: add to beg: One of the most important pieces in this collection is a marquetry commode by the éb

## Temporal

In [33]:
data, labels = to_datalabels(editor.template(
    {
        'contexts': [
            'Both {first_name} and {first_name2} were {prof1}s, but there was a change in {first_name}, who is now {a:prof2}.',
            'Both {first_name2} and {first_name} were {prof1}s, but there was a change in {first_name}, who is now {a:prof2}.',
        ],
        'qas': [
            (
                'Who is {a:prof2}?',
                '{first_name}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=100,
    ))
test = Mft(data, expect=expect_squad, labels=labels)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)

Predicting 192 examples



Test cases:      96
Fails (rate):    44 (45.8%)

Example fails:
C: Both Nicole and Austin were cleaners, but there was a change in Nicole, who is now a musician.
Q: Who is a musician?
A: Nicole
P: Nicole and Austin were cleaners, but there was a change in Nicole


----
C: Both Emma and Hannah were DJs, but there was a change in Hannah, who is now a photographer.
Q: Who is a photographer?
A: Hannah
P: Hannah were DJs, but there was a change in Hannah


----
C: Both Sara and Dylan were psychologists, but there was a change in Sara, who is now an artist.
Q: Who is an artist?
A: Sara
P: Sara and Dylan were psychologists, but there was a change in Sara


----


In [34]:
data, labels = to_datalabels(editor.template(
    {
        'contexts': [
            '{first_name} became a {prof} before {first_name2} did.',
            '{first_name2} became a {prof} after {first_name} did.',
        ],
        'qas': [
            (
                'Who became a {prof} first?',
                '{first_name}'
            ), 
            (
                'Who became a {prof} last?',
                '{first_name2}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=100,
    ))
test = Mft(data, expect=expect_squad, labels=labels)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)


Predicting 392 examples



Test cases:      98
Fails (rate):    82 (83.7%)

Example fails:
C: Jose became a analyst after Richard did.
Q: Who became a analyst last?
A: Jose
P: Richard


----
C: Danielle became a historian before James did.
Q: Who became a historian last?
A: James
P: Danielle


----
C: Aaron became a actress before Jeremy did.
Q: Who became a actress last?
A: Jeremy
P: Aaron

C: Jeremy became a actress after Aaron did.
Q: Who became a actress first?
A: Aaron
P: Jeremy


----


## Negation

In context

In [38]:
data, labels = to_datalabels(editor.template(
    {
        'contexts': [
            '{first_name} is not {a:prof}. {first_name2} is.',
            '{first_name2} is {a:prof}. {first_name} is not.',
        ],
        'qas': [
            (
                'Who is {a:prof}?',
                '{first_name2}'
            ), 
            (
                'Who is not {a:prof}?',
                '{first_name}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=200,
    ))
test = Mft(data, expect=expect_squad, labels=labels)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)

Predicting 792 examples



Test cases:      198
Fails (rate):    153 (77.3%)

Example fails:
C: Kelly is not an investigator. Sophia is.
Q: Who is an investigator?
A: Sophia
P: Kelly


----
C: Stephanie is a model. Christian is not.
Q: Who is not a model?
A: Christian
P: Stephanie


----
C: Jacob is an artist. Katherine is not.
Q: Who is not an artist?
A: Katherine
P: Jacob


----


Not in context:

In [39]:

data, labels = to_datalabels(editor.template(
    {
        'contexts': [
            '{first_name} is {a:prof}. {first_name2} is {a:prof2}.',
            '{first_name2} is {a:prof2}. {first_name} is {a:prof}.',
        ],
        'qas': [
            (
                'Who is {a:prof}?',
                '{first_name}'
            ), 
            (
                'Who is not {a:prof}?',
                '{first_name2}'
            ), 
            (
                'Who is {a:prof2}?',
                '{first_name2}'
            ), 
            (
                'Who is not {a:prof2}?',
                '{first_name}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=100,
    ))
test = Mft(data, expect=expect_squad, labels=labels)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)

Predicting 776 examples



Test cases:      97
Fails (rate):    97 (100.0%)

Example fails:
C: Andrea is a doctor. Amy is a DJ.
Q: Who is not a doctor?
A: Amy
P: Andrea

C: Andrea is a doctor. Amy is a DJ.
Q: Who is not a DJ?
A: Andrea
P: Amy

C: Amy is a DJ. Andrea is a doctor.
Q: Who is not a doctor?
A: Amy
P: Andrea


----
C: Isabella is an advisor. Jacob is a chef.
Q: Who is not an advisor?
A: Jacob
P: Isabella

C: Isabella is an advisor. Jacob is a chef.
Q: Who is not a chef?
A: Isabella
P: Jacob

C: Jacob is a chef. Isabella is an advisor.
Q: Who is not an advisor?
A: Jacob
P: Isabella


----
C: Anthony is an architect. Noah is an educator.
Q: Who is not an architect?
A: Noah
P: Anthony

C: Anthony is an architect. Noah is an educator.
Q: Who is not an educator?
A: Anthony
P: Noah

C: Noah is an educator. Anthony is an architect.
Q: Who is not an architect?
A: Noah
P: Anthony


----


## Fairness spinoff

bin           fail_men fail_women (count)
doctor          0.02      0.85     (104)
nurse           0.56      0.41     (90)
secretary       0.54      0.08     (106)


In [267]:
# fewer_profs = ['doctor', 'nurse', 'secretary']
datameta = editor.template(
    [
        ('{male} is not {a:prof}, {female} is.', 'Who is {a:prof}?', '{female}', 'woman', '{prof}'),
        ('{female} is not {a:prof}, {male} is.', 'Who is {a:prof}?', '{male}', 'man', '{prof}'),
    ],
    prof=professions,
#     prof=fewer_profs,
    remove_duplicates=True,
    nsamples=10000,
    unroll=True,
    )
data = [(d[0], d[1]) for d in datameta]
labels = [d[2] for d in datameta]
meta = [(d[3], d[4]) for d in datameta]

test = Mft(data, expect=expect_squad, labels=labels, meta=meta)
test.run(new_pp)

def print_fair(test):
    c = collections.Counter(test.meta)
    fail = collections.Counter([tuple(x) for x in np.array(test.meta)[test.fail_idxs()]])
    profs = set()
    for sex, prof in fail:
        profs.add(prof)
    prof_fail = {}
    get_fail = lambda f:fail[f] / c[f]
    for prof in profs:
        fail_m = get_fail(('man', prof))
        fail_f = get_fail(('woman', prof))
        prof_fail[prof] = (fail_m, fail_f)
    print('%-13s fail_men fail_women (count)' % 'profession')
    for prof, vs in sorted(prof_fail.items(), key=lambda x:max(x[1][0], x[1][1]), reverse=True):
        fail_m, fail_f = vs
        print('%-13s   %.2f      %.2f     (%d)' % (prof, fail_m, fail_f, c[('man', prof)]))
print_fair(test)

profession    fail_men fail_women (count)
doctor          0.03      0.87     (173)
secretary       0.62      0.02     (165)
nurse           0.61      0.35     (162)


## Coref

Basic coref

In [41]:
data, labels = to_datalabels(editor.template(
    {
        'contexts': [
            '{male} and {female} are friends. He is {a:prof1}, and she is {a:prof2}.',
            '{female} and {male} are friends. He is {a:prof1}, and she is {a:prof2}.',
            '{male} and {female} are friends. She is {a:prof2}, and he is {a:prof1}.',
            '{female} and {male} are friends. She is {a:prof2}, and he is {a:prof1}.',
        ],
        'qas': [
            (
                'Who is {a:prof1}?',
                '{male}'
            ), 
            (
                'Who is {a:prof2}?',
                '{female}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=100,
    ))
test = Mft(data, expect=expect_squad, labels=labels)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)

Predicting 784 examples



Test cases:      98
Fails (rate):    98 (100.0%)

Example fails:
C: Nathan and Andrea are friends. He is an entrepreneur, and she is an agent.
Q: Who is an entrepreneur?
A: Nathan
P: Nathan and Andrea

C: Andrea and Nathan are friends. He is an entrepreneur, and she is an agent.
Q: Who is an agent?
A: Andrea
P: Nathan

C: Nathan and Andrea are friends. She is an agent, and he is an entrepreneur.
Q: Who is an entrepreneur?
A: Nathan
P: Andrea


----
C: Richard and Natalie are friends. He is a waitress, and she is a photographer.
Q: Who is a waitress?
A: Richard
P: Natalie

C: Natalie and Richard are friends. He is a waitress, and she is a photographer.
Q: Who is a photographer?
A: Natalie
P: Richard

C: Richard and Natalie are friends. She is a photographer, and he is a waitress.
Q: Who is a waitress?
A: Richard
P: Natalie


----
C: Ashley and Gabriel are friends. He is a psychologist, and she is a model.
Q: Who is a model?
A: Ashley
P: Gabriel

C: Gabriel and Ashley are friends. She i

Former, latter

In [29]:
data, labels = to_datalabels(editor.template(
    {
        'contexts': [
            '{first_name} and {first_name2} are friends. The former is {a:prof1}.',
            '{first_name2} and {first_name} are friends. The latter is {a:prof1}.',
            '{first_name} and {first_name2} are friends. The former is {a:prof1} and the latter is {a:prof2}.',
            '{first_name2} and {first_name} are friends. The former is {a:prof2} and the latter is {a:prof1}.',
        ],
        'qas': [
            (
                'Who is {a:prof1}?',
                '{first_name}'
            ), 
        ]
        
    },
    prof=professions,
    remove_duplicates=True,
    nsamples=100,
    ))
test = Mft(data, expect=expect_squad, labels=labels)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)

Predicting 396 examples



Test cases:      99
Fails (rate):    99 (100.0%)

Example fails:
C: Jonathan and Jeremy are friends. The former is a photographer.
Q: Who is a photographer?
A: Jonathan
P: Jeremy

C: Jonathan and Jeremy are friends. The former is a photographer and the latter is a producer.
Q: Who is a photographer?
A: Jonathan
P: Jonathan and Jeremy


----
C: Steven and Benjamin are friends. The former is a therapist.
Q: Who is a therapist?
A: Steven
P: Benjamin

C: Steven and Benjamin are friends. The former is a therapist and the latter is a salesman.
Q: Who is a therapist?
A: Steven
P: Benjamin


----
C: Kimberly and Katherine are friends. The former is a cleaner.
Q: Who is a cleaner?
A: Kimberly
P: Katherine

C: Kimberly and Katherine are friends. The former is a cleaner and the latter is an investigator.
Q: Who is a cleaner?
A: Kimberly
P: Kimberly and Katherine


----


## SRL

In [53]:
import pattern
import pattern.en
pverb = ['love', 'hate', 'like', 'remember', 'recognize', 'trust', 'deserve', 'understand', 'blame', 'dislike', 'prefer', 'follow', 'notice', 'hurt', 'bother', 'support', 'believe', 'accept', 'attack']
a = pattern.en.tenses('loves')[0]
b = pattern.en.tenses('stolen')[0]
pverb = [(pattern.en.conjugate(v, *a), pattern.en.conjugate(v, *b)) for v in pverb]

data, labels = to_datalabels(editor.template(
    {
        'contexts': [
            '{first_name} {v[0]} {first_name2}.',
            '{first_name2} is {v[1]} by {first_name}.',
        ],
        'qas': [
            (
                'Who {v[0]}?',
                '{first_name}'
            ), 
            (
                'Who is {v[1]}?',
                '{first_name2}'
            ), 
        ]
        
    },
    v=pverb,
    remove_duplicates=True,
    nsamples=100,
    ))
test = Mft(data, expect=expect_squad, labels=labels)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)

Predicting 392 examples



Test cases:      98
Fails (rate):    54 (55.1%)

Example fails:
C: Samantha supports Michael.
Q: Who is supported?
A: Michael
P: Samantha supports Michael

C: Michael is supported by Samantha.
Q: Who is supported?
A: Michael
P: Samantha


----
C: Scott supports Danielle.
Q: Who is supported?
A: Danielle
P: Scott


----
C: Sophia trusts Kyle.
Q: Who trusts?
A: Sophia
P: Sophia trusts Kyle


----


In [55]:
data, labels = to_datalabels(editor.template(
    {
        'contexts': [
            '{first_name} {v[0]} {first_name2}. {first_name2} {v[0]} {first_name3}.',
            '{first_name} {v[0]} {first_name2}. {first_name3} is {v[1]} by {first_name2}.',
            '{first_name2} is {v[1]} by {first_name}. {first_name2} {v[0]} {first_name3}.',
            '{first_name2} is {v[1]} by {first_name}. {first_name3} is {v[1]} by {first_name2}.',
        ],
        'qas': [
            (
                'Who {v[0]} {first_name2}?',
                '{first_name}'
            ), 
            (
                'Who {v[0]} {first_name3}?',
                '{first_name2}'
            ), 
            (
                'Who is {v[1]} by {first_name}?',
                '{first_name2}'
            ), 
            (
                'Who is {v[1]} by {first_name2}?',
                '{first_name3}'
            ), 
        ]
        
    },
    v=pverb,
    remove_duplicates=True,
    nsamples=100,
    ))
test = Mft(data, expect=expect_squad, labels=labels)
test.run(new_pp)
test.summary(n=3, format_example_fn=format_squad_with_context)


Predicting 1568 examples



Test cases:      98
Fails (rate):    92 (93.9%)

Example fails:
C: Sophia accepts Joseph. Zachary is accepted by Joseph.
Q: Who accepts Zachary?
A: Joseph
P: Sophia accepts Joseph


----
C: Kelly understands Ethan. Nathan is understood by Ethan.
Q: Who understands Nathan?
A: Ethan
P: Kelly

C: Kelly understands Ethan. Nathan is understood by Ethan.
Q: Who is understood by Kelly?
A: Ethan
P: Nathan


----
C: Erin follows Jose. Michael is followed by Jose.
Q: Who is followed by Erin?
A: Jose
P: Jose. Michael

C: Jose is followed by Erin. Jose follows Michael.
Q: Who is followed by Jose?
A: Michael
P: Erin


----
